Install and import necessary libraries

In [33]:
%%capture
!pip install arch
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr

In [34]:
# Create dictionary for stock labels/tickers
sectors_dict = {
    "s&p" : "SPY",
    "communication_services" : "XLC",
    "consumer_discretionary" : "XLY",
    "consumer_staples" : "XLP",
    "energy" : "XLE",
    "financials" : "XLF",
    "healthcare" : "XLV",
    "industrials" : "XLI",
    "materials" : "XLB",
    "real_estate" : "XLRE",
    "technology" : "XLK",
    "utilities" : "XLU",
}

dataframe_dict = {}
close_dfs = []
start = "2021-01-01"
end = "2025-09-01"

for name, ticker in sectors_dict.items():
    hist = yf.Ticker(ticker).history(start=start, end=end,auto_adjust=True)

    # Keep only Adjusted Close and rename column to sector name
    df = hist[["Close"]].rename(columns={"Close": ticker})

    # Store cleaned df
    dataframe_dict[name] = df

    # Append for merging later
    close_dfs.append(df)

# Merge everything into a single DataFrame
sectors_close_df = pd.concat(close_dfs, axis=1)
sectors_close_df.index = sectors_close_df.index.tz_localize(None).date

Pulling data of external factors to stock volatility

In [35]:
# Download VIX (Adj Close)
vix_hist = yf.Ticker("^VIX").history(start=start, end=end, auto_adjust=True)
vix_df = vix_hist[["Close"]].rename(columns={"Close": "VIX"})

# Clean index: keep date only
vix_df.index = pd.to_datetime(vix_df.index).tz_localize(None).date

# Merge VIX with sectors df
yfinance_df = pd.concat([sectors_close_df, vix_df], axis=1)

fred_series = {
    'FEDFUNDS': 'Overnight Rate',     # monthly
    'DTB3': '1-Month Rate',           # daily
    'DTB6': '3-Month Rate',           # daily
    "CPIAUCSL": "CPI",                # monthly
    "PPIACO": "PPI",                  # monthly
    "UNRATE": "Unemployment Rate",    # monthly
    "GDP": "GDP"                      # quarterly
}

fred_data = pd.DataFrame()

for code, name in fred_series.items():
    series = pdr.DataReader(code, "fred", start, end)
    series.rename(columns={code: name}, inplace=True)
    fred_data = pd.concat([fred_data, series], axis=1)

# Make sure index is DatetimeIndex
fred_data.index = pd.to_datetime(fred_data.index)

# merge yfinance data with fred_date
merged = pd.merge(yfinance_df, fred_data,
                 left_index=True, right_index=True,
                 how='outer')

# filing in NaN data with last recorded data as sources have different reporting cadence
merged.ffill(inplace=True)
merged.reset_index(inplace=True)
merged['index'] = pd.to_datetime(merged['index'])
merged.rename(columns={'index': 'Date'}, inplace=True)

Data pre-processing and merging dataframes

In [36]:
df = merged.copy()
# Define sector columns
sector_cols = df.columns[1:13].tolist()

# Forward fill missing prices
df[sector_cols] = df[sector_cols].ffill()

# Calculate log returns for each sector column
# log returns = ln(P_t / P_{t-1})
for col in sector_cols:
    return_col_name = f'{col}_log_return'
    df[return_col_name] = np.log(df[col] / df[col].shift(1))


def rolling_realized_volatility(returns, window=5, annualize=False):
    """
    Calculate rolling realized volatility for log returns
    FORMULA: Realized volatility = sqrt(sum(r_t^2))

    Parameters:
    returns: log returns calculated above
    window: rolling window size, typically
    annualize: whether to annualize the result
    """
    # Calculate squared returns
    squared_returns = returns**2

    # Rolling sum of squared returns
    rolling_sum_sq = squared_returns.rolling(
        window=window,
        # min_periods=max(5, window//3)
    ).sum()

    # Realized volatility
    realized_vol = np.sqrt(rolling_sum_sq)

    # Annualize the realized volatility
    if annualize:
        # Annualize: √(252/window * sum(r_t²))
        realized_vol = realized_vol * np.sqrt(252/window)

    return realized_vol

# Calculate realized volatility for each sector and add it as a new column
for col in sector_cols:
    return_col_name = f'{col}_log_return'
    vol_col_name = f'{col}_vol_5d'

    # Use the log returns column for volatility calculation, not the price column
    df[vol_col_name] = rolling_realized_volatility(df[return_col_name])

# Reorder columns to place volatility after each return column
new_columns_order = []

# Add all non-derived columns first
for col in df.columns:
    if not any([col.endswith('_log_return'), col.endswith('_vol_5d')]):
        new_columns_order.append(col)

# Now add derived columns in the desired order
for col in sector_cols:
    return_col = f'{col}_log_return'
    vol_col = f'{col}_vol_5d'

    # Find the position of the original price column
    if col in new_columns_order:
        col_idx = new_columns_order.index(col)

        # Insert return column after price column
        if return_col in df.columns:
            new_columns_order.insert(col_idx + 1, return_col)

            # Insert volatility column after return column
            if vol_col in df.columns:
                new_columns_order.insert(col_idx + 2, vol_col)
        else:
            # Insert volatility column after price column if no return column
            if vol_col in df.columns:
                new_columns_order.insert(col_idx + 1, vol_col)

# Reorder the dataframe
df = df[new_columns_order]

In [37]:
# Create a copy
df_pct = df.copy()
df_pct = df_pct.set_index('Date').sort_index()

# Calculate monthly percentage changes for CPI and PPI
monthly_first = df_pct[['CPI', 'PPI']].resample('MS').first()
monthly_pct = monthly_first.pct_change() * 100
monthly_pct.columns = ['CPI_pct_change', 'PPI_pct_change']

# Calculate quarterly percentage changes for GDP
quarterly_first = df_pct[['GDP']].resample('QS').first()
quarterly_pct = quarterly_first.pct_change() * 100
quarterly_pct.columns = ['GDP_pct_change']

all_pct_changes = pd.concat([monthly_pct, quarterly_pct], axis=1)

# Convert to monthly/quarterly pct to daily and ffill
start_date = df_pct.index.min()
end_date = df_pct.index.max()
full_daily_index = pd.date_range(start=start_date, end=end_date, freq='D')

daily_pct_changes = all_pct_changes.reindex(full_daily_index).ffill()

# Merge with original dataframe
df = df.merge(daily_pct_changes, left_on='Date', right_index=True, how='left')

# Calculate VIX log
vix = df['VIX']
vix_log = np.log(vix / vix.shift(1)) * 100
vix_position = df.columns.get_loc('VIX') + 1
if 'VIX_log' not in df.columns:
    df.insert(vix_position, 'VIX_log', vix_log)

In [38]:
# Fill no percentage change with 0 and dropna
df['CPI_pct_change'] = df['CPI_pct_change'].fillna(0)
df['PPI_pct_change'] = df['PPI_pct_change'].fillna(0)
df['GDP_pct_change'] = df['GDP_pct_change'].fillna(0)
df.dropna(inplace=True)

Export dataframe to file and preview first 5 rows

In [39]:
df.to_csv('data.csv')
df.head()

,Date,SPY,SPY_log_return,SPY_vol_5d,XLC,XLC_log_return,XLC_vol_5d,XLY,XLY_log_return,XLY_vol_5d,...,Overnight Rate,1-Month Rate,3-Month Rate,CPI,PPI,Unemployment Rate,GDP,CPI_pct_change,PPI_pct_change,GDP_pct_change
6,2021-01-11,354.542755,-0.006763,0.019447,63.310917,-0.017940,0.023259,159.420868,-0.018838,0.035987,...,0.09,0.08,0.10,262.639,204.8,6.4,22680.693,0.0,0.0,0.0
7,2021-01-12,354.617584,0.000211,0.018196,62.289005,-0.016273,0.028157,161.855865,0.015159,0.038334,...,0.09,0.09,0.09,262.639,204.8,6.4,22680.693,0.0,0.0,0.0
8,2021-01-13,355.572571,0.002689,0.017402,62.508682,0.003521,0.027966,161.846252,-0.000059,0.036964,...,0.09,0.09,0.10,262.639,204.8,6.4,22680.693,0.0,0.0,0.0
9,2021-01-14,354.327332,-0.003508,0.009879,62.155300,-0.005669,0.025316,160.816376,-0.006384,0.032279,...,0.09,0.09,0.09,262.639,204.8,6.4,22680.693,0.0,0.0,0.0
10,2021-01-15,351.743347,-0.007319,0.010904,62.355869,0.003222,0.025329,159.574829,-0.007750,0.026182,...,0.09,0.09,0.10,262.639,204.8,6.4,22680.693,0.0,0.0,0.0
